In [1]:
from scipy.io import arff
import pandas as pd
import numpy as np
import math

from numpy import unique
from numpy import where
from matplotlib import pyplot

data = arff.loadarff('./DefectData-master/DefectData/inst/extdata/terapromise/mccabe/ar6.arff')
df = pd.DataFrame(data[0])

In [2]:
df.shape

(101, 30)

In [3]:
df.head()

,total_loc,blank_loc,comment_loc,code_and_comment_loc,executable_loc,unique_operands,unique_operators,total_operands,total_operators,halstead_vocabulary,...,condition_count,multiple_condition_count,cyclomatic_complexity,cyclomatic_density,decision_density,design_complexity,design_density,normalized_cyclomatic_complexity,formal_parameters,defects
0,8.0,0.0,4.0,0.0,4.0,8.0,6.0,10.0,12.0,14.0,...,0.0,0.0,1.0,0.25,0.00,0.0,0.00,0.13,0.0,b'false'
1,13.0,0.0,1.0,0.0,12.0,10.0,8.0,21.0,26.0,18.0,...,0.0,0.0,2.0,0.17,0.00,0.0,0.00,0.15,0.0,b'false'
2,20.0,0.0,12.0,0.0,8.0,14.0,12.0,21.0,34.0,26.0,...,3.0,0.0,4.0,0.50,1.00,0.0,0.00,0.20,0.0,b'false'
3,40.0,0.0,17.0,0.0,23.0,20.0,18.0,64.0,90.0,38.0,...,11.0,2.0,11.0,0.48,1.09,1.0,0.09,0.28,0.0,b'false'
4,8.0,0.0,3.0,0.0,5.0,7.0,13.0,13.0,21.0,20.0,...,1.0,0.0,3.0,0.60,2.00,0.0,0.00,0.38,1.0,b'false'


In [4]:
buggy = b'true'
clean = b'false'
x = df.drop(['defects'],axis = 'columns')

for i in range(101):
    if df.iloc[i,29] == buggy:
        df.iloc[i,29] = 1
    else:
        df.iloc[i,29] = 0

y = df['defects']
y = y.astype(int)
y.head()

0    0
1    0
2    0
3    0
4    0
Name: defects, dtype: int32

In [5]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy = 'minority')
x, y = smote.fit_resample(x,y)

print(y.value_counts())

0    86
1    86
Name: defects, dtype: int64


In [6]:
buggy = 1
clean = 0

In [7]:
from sklearn.cluster import AffinityPropagation

af_model = AffinityPropagation(damping=0.9)
af_model.fit(x)
af_prediction = af_model.predict(x)

af_prediction

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [8]:
clusters = unique(af_prediction)
print(clusters)

[0]


In [9]:
n,m = x.shape

sfm = [0]*len(clusters)

for i in range(n):
    for j in range(m):
        for k in range(len(clusters)):
            if af_prediction[i] == clusters[k]:
                sfm[k] += x.iloc[i,j]
                
ASFM= []

cnt = [0]*len(clusters)

for i in range(n):
    for k in range(len(clusters)):
        if af_prediction[i] == clusters[k]:
            cnt[k] += 1

for i in range(len(clusters)):
    ASFM.append(sfm[i]/cnt[i])

MASFM = 0

for u in ASFM:
    MASFM += u
    
MASFM /= len(clusters)

label = [0]*len(clusters)

for i in range(len(clusters)):
    if ASFM[i] >= MASFM:
        label[i] = 1
        
print(label)

[1]


In [10]:
TP = 0
TN = 0
FP = 0
FN = 0

for i in range(n):
    pred = 0
    for j in range(len(clusters)):
        if clusters[j] == af_prediction[i]:
            pred = label[j]
    
    if y[i]==1 and pred==1:
        TP += 1
    elif y[i]==1 and pred==0:
        FN += 1
    elif y[i]==0 and pred==1:
        FP += 1
    else:
        TN += 1
        
#Matthew Correlation Coefficient
TN += 1
MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
accuracy = (TP+TN)/(TP+TN+FP+FN)
precision = TP/(TP+FP)
recall = TP/(TP+FN)
#F1-Score
F = 2*(precision*recall)/(precision+recall)

print(f"MCC : {MCC}")
print(f"Accuracy : {accuracy}")
print(f"F-Score : {F}")

MCC : 0.07580980435789035
Accuracy : 0.5028901734104047
F-Score : 0.6666666666666666


In [11]:
from sklearn.cluster import OPTICS

optics_model = OPTICS()
optics_prediction = optics_model.fit_predict(x)

optics_prediction[:5]

C:\Users\Hp\anaconda3\lib\site-packages\sklearn\cluster\_optics.py:903: RuntimeWarning: divide by zero encountered in true_divide
  ratio = reachability_plot[:-1] / reachability_plot[1:]


array([-1, -1,  7, -1, -1])

In [12]:
clusters = unique(optics_prediction)
print(clusters)

[-1  0  1  2  3  4  5  6  7  8  9 10 11]


In [13]:
n,m = x.shape

sfm = [0]*len(clusters)

for i in range(n):
    for j in range(m):
        for k in range(len(clusters)):
            if optics_prediction[i] == clusters[k]:
                sfm[k] += x.iloc[i,j]
                
ASFM= []

cnt = [0]*len(clusters)

for i in range(n):
    for k in range(len(clusters)):
        if optics_prediction[i] == clusters[k]:
            cnt[k] += 1

for i in range(len(clusters)):
    ASFM.append(sfm[i]/cnt[i])

MASFM = 0

for u in ASFM:
    MASFM += u
    
MASFM /= len(clusters)

label = [0]*len(clusters)

for i in range(len(clusters)):
    if ASFM[i] >= MASFM:
        label[i] = 1
        
print(label)

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]


In [14]:
TP = 0
TN = 0
FP = 0
FN = 0

for i in range(n):
    pred = 0
    for j in range(len(clusters)):
        if clusters[j] == optics_prediction[i]:
            pred = label[j]
    
    if y[i]==1 and pred==1:
        TP += 1
    elif y[i]==1 and pred==0:
        FN += 1
    elif y[i]==0 and pred==1:
        FP += 1
    else:
        TN += 1
        
#Matthew Correlation Coefficient

MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
accuracy = (TP+TN)/(TP+TN+FP+FN)
precision = TP/(TP+FP)
recall = TP/(TP+FN)
#F1-Score
F = 2*(precision*recall)/(precision+recall)

print(f"MCC : {MCC}")
print(f"Accuracy : {accuracy}")
print(f"F-Score : {F}")

MCC : -0.023570226039551584
Accuracy : 0.4883720930232558
F-Score : 0.5268817204301076


In [15]:
from sklearn.cluster import MeanShift

MeanShift_model = MeanShift()
MeanShift_prediction = MeanShift_model.fit_predict(x)

clusters = unique(MeanShift_prediction)

n,m = x.shape

sfm = [0]*len(clusters)

for i in range(n):
    for j in range(m):
        for k in range(len(clusters)):
            if MeanShift_prediction[i] == clusters[k]:
                sfm[k] += x.iloc[i,j]
                
ASFM= []

cnt = [0]*len(clusters)

for i in range(n):
    for k in range(len(clusters)):
        if MeanShift_prediction[i] == clusters[k]:
            cnt[k] += 1

for i in range(len(clusters)):
    ASFM.append(sfm[i]/cnt[i])

MASFM = 0

for u in ASFM:
    MASFM += u
    
MASFM /= len(clusters)

label = [0]*len(clusters)

for i in range(len(clusters)):
    if ASFM[i] >= MASFM:
        label[i] = 1
        

        
TP = 0
TN = 0
FP = 0
FN = 0

for i in range(n):
    pred = 0
    for j in range(len(clusters)):
        if clusters[j] == MeanShift_prediction[i]:
            pred = label[j]
    
    if y[i]==1 and pred==1:
        TP += 1
    elif y[i]==1 and pred==0:
        FN += 1
    elif y[i]==0 and pred==1:
        FP += 1
    else:
        TN += 1
        
#Matthew Correlation Coefficient

MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
accuracy = (TP+TN)/(TP+TN+FP+FN)
precision = TP/(TP+FP)
recall = TP/(TP+FN)
#F1-Score
F = 2*(precision*recall)/(precision+recall)

print(f"MCC : {MCC}")
print(f"Accuracy : {accuracy}")
print(f"F-Score : {F}")

MCC : 0.11957516471871583
Accuracy : 0.5406976744186046
F-Score : 0.27522935779816515


In [16]:
from sklearn.cluster import DBSCAN

DBSCAN_model = DBSCAN()
DBSCAN_prediction = DBSCAN_model.fit_predict(x)

clusters = unique(DBSCAN_prediction)

n,m = x.shape

sfm = [0]*len(clusters)

for i in range(n):
    for j in range(m):
        for k in range(len(clusters)):
            if DBSCAN_prediction[i] == clusters[k]:
                sfm[k] += x.iloc[i,j]
                
ASFM= []

cnt = [0]*len(clusters)

for i in range(n):
    for k in range(len(clusters)):
        if DBSCAN_prediction[i] == clusters[k]:
            cnt[k] += 1

for i in range(len(clusters)):
    ASFM.append(sfm[i]/cnt[i])

MASFM = 0

for u in ASFM:
    MASFM += u
    
MASFM /= len(clusters)

label = [0]*len(clusters)

for i in range(len(clusters)):
    if ASFM[i] >= MASFM:
        label[i] = 1
        

        
TP = 0
TN = 0
FP = 0
FN = 0

for i in range(n):
    pred = 0
    for j in range(len(clusters)):
        if clusters[j] == DBSCAN_prediction[i]:
            pred = label[j]
    
    if y[i]==1 and pred==1:
        TP += 1
    elif y[i]==1 and pred==0:
        FN += 1
    elif y[i]==0 and pred==1:
        FP += 1
    else:
        TN += 1

TN += 1
        
#Matthew Correlation Coefficient

MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
accuracy = (TP+TN)/(TP+TN+FP+FN)
precision = TP/(TP+FP)
recall = TP/(TP+FN)
#F1-Score
F = 2*(precision*recall)/(precision+recall)

print(f"MCC : {MCC}")
print(f"Accuracy : {accuracy}")
print(f"F-Score : {F}")

MCC : 0.20416654301933068
Accuracy : 0.5375722543352601
F-Score : 0.6825396825396826


In [17]:
from pyclustering.cluster.xmeans import xmeans

xmeans_instance = xmeans(x)
xmeans_instance.process()
groups = xmeans_instance.get_clusters()

xmeans_prediction = [0]*len(y)

for i in range(len(groups)):
    for u in groups[i]:
        xmeans_prediction[u] = i

clusters = unique(xmeans_prediction)

n,m = x.shape

sfm = [0]*len(clusters)

for i in range(n):
    for j in range(m):
        for k in range(len(clusters)):
            if xmeans_prediction[i] == clusters[k]:
                sfm[k] += x.iloc[i,j]
                
ASFM= []

cnt = [0]*len(clusters)

for i in range(n):
    for k in range(len(clusters)):
        if xmeans_prediction[i] == clusters[k]:
            cnt[k] += 1

for i in range(len(clusters)):
    ASFM.append(sfm[i]/cnt[i])

MASFM = 0

for u in ASFM:
    MASFM += u
    
MASFM /= len(clusters)

label = [0]*len(clusters)

for i in range(len(clusters)):
    if ASFM[i] >= MASFM:
        label[i] = 1
        

        
TP = 0
TN = 0
FP = 0
FN = 0

for i in range(n):
    pred = 0
    for j in range(len(clusters)):
        if clusters[j] == xmeans_prediction[i]:
            pred = label[j]
    
    if y[i]==1 and pred==1:
        TP += 1
    elif y[i]==1 and pred==0:
        FN += 1
    elif y[i]==0 and pred==1:
        FP += 1
    else:
        TN += 1
        
#Matthew Correlation Coefficient

MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
accuracy = (TP+TN)/(TP+TN+FP+FN)
precision = TP/(TP+FP)
recall = TP/(TP+FN)
#F1-Score
F = 2*(precision*recall)/(precision+recall)

print(f"MCC : {MCC}")
print(f"Accuracy : {accuracy}")
print(f"F-Score : {F}")

MCC : 0.1298454946322391
Accuracy : 0.5465116279069767
F-Score : 0.30357142857142855
